In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_karkidi_jobs(keyword="data science", pages=1):
    headers = {'User-Agent': 'Mozilla/5.0'}
    base_url = "https://www.karkidi.com/Find-Jobs/{page}/all/India?search={query}"
    jobs_list = []

    for page in range(1, pages + 1):
        url = base_url.format(page=page, query=keyword.replace(' ', '%20'))
        print(f"Scraping page: {page}")
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        job_blocks = soup.find_all("div", class_="ads-details")
        for job in job_blocks:
            try:
                title = job.find("h4").get_text(strip=True)
                company = job.find("a", href=lambda x: x and "Employer-Profile" in x).get_text(strip=True)
                location = job.find("p").get_text(strip=True)
                experience = job.find("p", class_="emp-exp").get_text(strip=True)
                key_skills_tag = job.find("span", string="Key Skills")
                skills = key_skills_tag.find_next("p").get_text(strip=True) if key_skills_tag else ""
                summary_tag = job.find("span", string="Summary")
                summary = summary_tag.find_next("p").get_text(strip=True) if summary_tag else ""

                jobs_list.append({
                    "Title": title,
                    "Company": company,
                    "Location": location,
                    "Experience": experience,
                    "Summary": summary,
                    "Skills": skills
                })
            except Exception as e:
                print(f"Error parsing job block: {e}")
                continue

        time.sleep(1)  # Be nice to the server

    return pd.DataFrame(jobs_list)

# Example use:
if __name__ == "__main__":
    df_jobs = scrape_karkidi_jobs(keyword="data science", pages=2)
    print(df_jobs.head())


Scraping page: 1
Scraping page: 2
                                               Title         Company  \
0  Staff Software Engineer - Monetization, Poe (R...     Quora, Inc.   
1  Staff Backend Engineer - Bot Creator Ecosystem...     Quora, Inc.   
2  Senior Backend Engineer - Bot Creator Ecosyste...     Quora, Inc.   
3                         Data Scientist Lead - AIML  JPMorgan Chase   
4          Applied AI ML Director - Machine Learning  JPMorgan Chase   

                      Location  Experience  \
0                        India   8-10 year   
1                        India   8-10 year   
2                        India    6-8 year   
3  Bengaluru, Karnataka, India    6-8 year   
4  Hyderabad, Telangana, India  12-14 year   

                                             Summary  \
0  About Quora:Quora’s mission is to grow and sha...   
1  About Quora:Quora’s mission is to grow and sha...   
2  About Quora:Quora’s mission is to grow and sha...   
3  We have an opportunity to imp

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import joblib

# STEP 1: Preprocess skills text
def preprocess_skills(df):
    df['Skills'] = df['Skills'].fillna('').str.lower()
    return df

# STEP 2: Convert skills to TF-IDF vectors
def vectorize_skills(df):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(df['Skills'])
    return X, vectorizer

# STEP 3: Cluster using KMeans
def cluster_jobs(X, n_clusters=5):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X)
    return kmeans

# STEP 4: Save model and vectorizer
def save_model(kmeans, vectorizer):
    joblib.dump(kmeans, 'karkidi_kmeans_model.pkl')
    joblib.dump(vectorizer, 'karkidi_vectorizer.pkl')

# MAIN PIPELINE
if __name__ == "__main__":
    # Step 0: Scrape jobs (your existing function)
    #from your_scraper_file import scrape_karkidi_jobs  # replace with actual file if separate
    df_jobs = scrape_karkidi_jobs(keyword="data science", pages=2)

    # Step 1: Clean
    df_jobs = preprocess_skills(df_jobs)

    # Step 2: Vectorize
    X, vectorizer = vectorize_skills(df_jobs)

    # Step 3: Cluster
    kmeans = cluster_jobs(X, n_clusters=5)
    df_jobs['Cluster'] = kmeans.labels_

    # Step 4: Save
    save_model(kmeans, vectorizer)

    # Optional: Save DataFrame
    df_jobs.to_csv('clustered_jobs.csv', index=False)

    print(df_jobs[['Title', 'Company', 'Skills', 'Cluster']].head())


Scraping page: 1
Scraping page: 2
                                               Title         Company  \
0  Staff Software Engineer - Monetization, Poe (R...     Quora, Inc.   
1  Staff Backend Engineer - Bot Creator Ecosystem...     Quora, Inc.   
2  Senior Backend Engineer - Bot Creator Ecosyste...     Quora, Inc.   
3                         Data Scientist Lead - AIML  JPMorgan Chase   
4          Applied AI ML Director - Machine Learning  JPMorgan Chase   

                                              Skills  Cluster  
0  aartificial intelligence,analytical and proble...        1  
1  aartificial intelligence,api,data science tech...        1  
2  aartificial intelligence,api,data science tech...        1  
3  aartificial intelligence,data science techniqu...        2  
4  aartificial intelligence,aws,azure,google clou...        0  


In [ ]:
import joblib

# Load the saved KMeans model and vectorizer
kmeans = joblib.load('karkidi_kmeans_model.pkl')
vectorizer = joblib.load('karkidi_vectorizer.pkl')


In [ ]:
def classify_new_jobs(new_jobs_df, vectorizer, kmeans):
    # Preprocess the new jobs' skills column
    new_jobs_df['Skills'] = new_jobs_df['Skills'].fillna('').str.lower()

    # Vectorize using the same vectorizer
    X_new = vectorizer.transform(new_jobs_df['Skills'])

    # Predict cluster
    clusters = kmeans.predict(X_new)
    new_jobs_df['Cluster'] = clusters

    return new_jobs_df


In [ ]:
def alert_users(jobs_df, preferred_clusters):
    matched_jobs = jobs_df[jobs_df['Cluster'].isin(preferred_clusters)]
    if not matched_jobs.empty:
        print("\n📢 New jobs matching user interests:")
        print(matched_jobs[['Title', 'Company', 'Skills', 'Cluster']])
    else:
        print("\n❌ No new jobs found for the user's interests.")


In [ ]:
# Scrape fresh jobs
new_jobs_df = scrape_karkidi_jobs(keyword="data science", pages=1)

# Classify them
classified_jobs = classify_new_jobs(new_jobs_df, vectorizer, kmeans)

# Example: User is interested in clusters 1 and 3
user_clusters = [1, 3]
alert_users(classified_jobs, user_clusters)


Scraping page: 1

📢 New jobs matching user interests:
                                               Title      Company  \
0  Staff Software Engineer - Monetization, Poe (R...  Quora, Inc.   
1  Staff Backend Engineer - Bot Creator Ecosystem...  Quora, Inc.   
2  Senior Backend Engineer - Bot Creator Ecosyste...  Quora, Inc.   
5                            Senior Product Designer   Observe.AI   

                                              Skills  Cluster  
0  aartificial intelligence,analytical and proble...        1  
1  aartificial intelligence,api,data science tech...        1  
2  aartificial intelligence,api,data science tech...        1  
5  design,leadership skill,machine learning techn...        3  
